In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [ ]:
#wget https://springernature.figshare.com/ndownloader/files/3195389 -O dsgdb9nsd.xyz.tar.bz2
#!ls dsgdb9nsd.xyz/ | while read i; do sed -i 's/*^/E/g' dsgdb9nsd.xyz/$i; done

In [2]:
train_fname = Path('train.npz')
try:
    npzfile = np.load(train_fname)
    m = npzfile['m']
except:
    assert False

In [3]:
t = pd.read_csv('train.csv')

In [4]:
m_map = pd.factorize(t['molecule_name'])[1]

In [5]:
df = pd.DataFrame({'m': m_map[m]})

In [6]:
#df = df[:100]
df.head()

,m
0,dsgdb9nsd_000001
1,dsgdb9nsd_000001
2,dsgdb9nsd_000001
3,dsgdb9nsd_000001
4,dsgdb9nsd_000002


In [7]:
def apply(row):
    fname = 'dsgdb9nsd.xyz/%s.xyz' % row['m']
    zeros = np.zeros((1, 29), dtype=np.float32)
    qm = pd.read_csv(fname,
                     skiprows=2,
                     nrows=int(open(fname).readline().rstrip()),
                     sep='\t',
                     header=None)[[4]].values.T
    zeros[:, :qm.shape[1]] = qm
    return zeros


df['x_qm9_mulliken'] = df.progress_apply(apply, axis=1)

100%|██████████| 785836/785836 [42:46<00:00, 306.21it/s] 


In [8]:
df.head()

,m,x_qm9_mulliken
0,dsgdb9nsd_000001,"[[-0.535689, 0.133921, 0.133922, 0.133923, 0.1..."
1,dsgdb9nsd_000001,"[[-0.535689, 0.133921, 0.133922, 0.133923, 0.1..."
2,dsgdb9nsd_000001,"[[-0.535689, 0.133921, 0.133922, 0.133923, 0.1..."
3,dsgdb9nsd_000001,"[[-0.535689, 0.133921, 0.133922, 0.133923, 0.1..."
4,dsgdb9nsd_000002,"[[-0.707143, 0.235712, 0.235712, 0.23572, 0.0,..."


In [9]:
x_qm9_mulliken = df['x_qm9_mulliken'].values

In [10]:
x_qm9_mulliken = np.array(x_qm9_mulliken.tolist()).astype(np.float32)
np.save('x_qm9_mulliken.npy', x_qm9_mulliken)

In [13]:
from functools import partial
use_memmap = True
load_fn = np.load if not use_memmap else partial(np.lib.format.open_memmap,
                                                 mode='r')
x_qm9_mulliken = load_fn('x_qm9_mulliken.npy')
x_qm9_mulliken.shape

(785836, 1, 29)

In [15]:
!ls -altrh x_qm9_mulliken.npy

-rw-rw-r-- 1 ubuntu ubuntu 87M Jun 28 08:43 x_qm9_mulliken.npy


In [19]:
test_fname = Path('test.npz')
try:
    npzfile_test = np.load(test_fname)
    m_test = npzfile_test['m']
except:
    assert False
test = pd.read_csv('test.csv')
m_map_test = pd.factorize(test['molecule_name'])[1]
m_map_test
df_test = pd.DataFrame({'m': m_map_test[m_test - 4658147]})

In [20]:
df_test.head()

,m
0,dsgdb9nsd_000004
1,dsgdb9nsd_000004
2,dsgdb9nsd_000015
3,dsgdb9nsd_000015
4,dsgdb9nsd_000015


In [21]:
df_test['x_qm9_mulliken']=df_test.progress_apply(apply, axis=1)

100%|██████████| 422550/422550 [23:08<00:00, 304.42it/s]


In [22]:
xt_qm9_mulliken = df_test['x_qm9_mulliken'].values

In [23]:
xt_qm9_mulliken = np.array(xt_qm9_mulliken.tolist()).astype(np.float32)
np.save('xt_qm9_mulliken.npy', xt_qm9_mulliken)

In [24]:
from functools import partial
use_memmap = True
load_fn = np.load if not use_memmap else partial(np.lib.format.open_memmap, mode='r')
xt_qm9_mulliken = load_fn('xt_qm9_mulliken.npy')
xt_qm9_mulliken.shape

(422550, 1, 29)

In [26]:
!ls -altrh xt_qm9_mulliken.npy

-rw-rw-r-- 1 ubuntu ubuntu 47M Jun 28 09:15 xt_qm9_mulliken.npy


In [27]:
!a3 xt_qm9_mulliken.npy

https://s3-us-west-2.amazonaws.com/fyusion/temp//xt_qm9_mulliken.npy


In [28]:
!a3 x_qm9_mulliken.npy

https://s3-us-west-2.amazonaws.com/fyusion/temp//x_qm9_mulliken.npy
